In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules, apriori
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from sklearn.manifold import TSNE

# 1
Загрузите файл «assc_TRANSACTION.csv». В нем три колонки Customer – ID клиента, Product –
покупка, Time – временная метка (для задания не нужна). Определите (написав
соответствующий код) сколько различных значений принимают переменные Product и
Customer.

In [ ]:
df = pd.read_csv("assc_TRANSACTION.csv")
del df["TIME"]
df.nunique()

# 2
Найдите частые эпизоды с ограничением на размер правила равным 4, с использованием
алгоритма и порога на поддержку согласно вашему варианту. (Apriori, 6%)

In [ ]:
# Формирование списка транзакций
transactions = df.groupby("CUSTOMER").aggregate({"PRODUCT":list}).values[:, 0]

# Кодирование транзакций
te = TransactionEncoder()
te_ary = te.fit_transform(transactions)
df2 = pd.DataFrame(te_ary, columns=te.columns_)

# Построение модели
frequent_itemsets = apriori(df2, min_support=0.06, use_colnames=True, max_len=4)
frequent_itemsets

# 3
Найдите самый большой (где больше всего элементов) частый эпизод, содержащий продукт
согласно вашему варианту. Какая у него поддержка? (soda)

In [ ]:
df3 = frequent_itemsets.copy()

df3 = df3[df3["itemsets"].astype(str).str.contains("soda")]

df3["length"] = df3["itemsets"].apply(lambda x: len(x))

df3.sort_values(by=["length", "support"], ascending=False).head(1)

# 4
На основе найденных частых эпизодов постройте ассоциативные правила с порогом на
достоверность согласно вашему варианту. Найдите правило с максимальным лифтом,
содержащем продукт из вашего варианта в левой части правила. Дайте ему письменную
словесную интерпретацию, укажите и объясните его числовые показатели: поддержку,
достоверность и подъем. (20%, "soda")

In [ ]:
df4 = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

df4[df4["antecedents"].astype(str).str.contains("soda")].sort_values(by=["lift"], ascending=False).head(1)

(soda, bourbon) $\to$ (olives, cracker)

Правило заключается в том, что при покупке (soda, bourbon) купят и (olives, cracker).

Поддержка $=0.107892$.

Поддержка - процент транзакций, содержащих оба набора атрибутов.

Достоверность $=0.696774$.

Достоверность - вероятность, что транзакция, содержащая 1-й набор атрибутов, будет содержать и 2-й. $1 \to 2$

Подъем $=3.770113$.

Подъем - отношение достоверности $(1 \to 2)$ к поддержке $(2)$.

$$
\operatorname{support} (X \implies Y) = P(X \cap Y)
\\[0.5 cm]
\operatorname{confidence} (X \implies Y) = P(Y | X)
\\[0.5 cm]
\operatorname{lift} (X \implies Y) = \frac {P(Y | X)} {P(Y)} = \frac {\operatorname{confidence} (X \implies Y)}
{P(Y)}
$$

# 5
Используя только двухместные правила постройте ориентированный граф, где вершины
элементы правила, их цвет (или размер) – поддержка элемента (item support), дуги –
импликации (ориентированы в направлении от условия к следствию), веса дугu –
достоверности.

In [ ]:
df5 = df4.copy()

df5["len1"] = df5["antecedents"].apply(lambda x: len(x))
df5["len2"] = df5["consequents"].apply(lambda x: len(x))

df5 = df5[(df5["len1"] == 1) & (df5["len2"] == 1)]

G = nx.from_pandas_edgelist(
    df5, source="antecedents", target="consequents", edge_attr="confidence", create_using=nx.DiGraph)

d = df5.groupby('antecedents')['antecedent support'].max().to_dict()

nx.set_node_attributes(G, d, "support")

nodes_weights = list(nx.get_node_attributes(G, 'support').values())

edge_weights = list(nx.get_edge_attributes(G, 'confidence').values())

nodes_weights = list(map(lambda x : x * 10000, nodes_weights))
edge_weights = list(map(lambda x : x * 5, edge_weights))



plt.figure(figsize=(16,9))

nx.draw_networkx(G, arrows=True,
                 node_size=nodes_weights, node_color=nodes_weights,
                 width=edge_weights, edge_color=edge_weights)
plt.show()

# 6
Для данного графа рассчитайте меры центральности согласно вашему варианту и найдите
элемент с самой высокой мерой, а также какую меру имеет продукт из вашего варианта. (Authority, soda)

In [ ]:
h, a = nx.hits(G)
metr = dict(sorted(a.items(), key=lambda x: x[1]))

print(f"Элемент с самой высокой мерой - {list(list(metr.keys())[-1])[0]}")
print(f"Мера soda - {metr[frozenset({'soda'})]}")

plt.xticks(rotation="vertical")
plt.bar(list(map(lambda x: list(x)[0], metr.keys())),  metr.values())
plt.show()

# 7
Постройте числовую матрицу со счетчиком числа покупок в ячейках, клиентами по строкам и
продуктами по столбцам.

In [ ]:
df7 = df.groupby("CUSTOMER")["PRODUCT"].value_counts().unstack(fill_value=0)
df7

# 8
С помощью метода из вашего варианта постройте линейную проекцию набора данных на
плоскость (2 компоненты) цветом укажите транзакции, содержащие продукт вашего варианта. (PCA, soda)

In [ ]:
pca = PCA(n_components=2)
features = pca.fit_transform(df7)

plt.scatter(features[:, 0], features[:, 1], c=df7["soda"])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# 9
С помощью метода из вашего варианта постройте нелинейную проекцию набора данных на
плоскость цветом укажите транзакции, содержащие продукт из вашего варианта. Не
указанные в задании параметры (например, размер решетки для SOM или число слоев в
автоэнкодере можно выбирать на свое усмотрение для получение наиболее удобной
визуализации). Дайте письменный комментарий, чем с вашей точки зрения для вашего
примера лучше или хуже нелинейная проекция). (AutoEncoder, soda)

In [ ]:
X = df7
Y = df7["soda"]

model = MLPRegressor(hidden_layer_sizes=(2, 2, 2), max_iter=1000, alpha=0.0001)

model.fit(X, X)

dummy = np.zeros((1, 2))

hidden = MLPRegressor(hidden_layer_sizes=[], max_iter=1000).fit(X[:1], dummy)

hidden.coefs_[0] = model.coefs_[0]

neurons = hidden.predict(X)

plt.scatter(*neurons.T, c=Y)
plt.xlabel("Neuron-1")
plt.ylabel("Neuron-2")
plt.show()

# embedded = TSNE(n_components=2, learning_rate="auto", init="random", perplexity=3).fit_transform(df7)
# plt.scatter(*embedded.T, c=df7["soda"])
# plt.show()

Линейная проекция в моем случае лучше, так как на ней более явно заметны группы.

# 10
Из исходной матрицы (из пункта 7) согласно вашему варианту отберите указанное число
независимых переменных с использованием заданного метода. (VarClus, 7)

In [ ]:
from varclushi import VarClusHi

clusters = VarClusHi(df7, maxeigval2=0.9, maxclus=7)
clusters.varclus()

max_RS_Ratio = clusters.rsquare.sort_values(by=["Cluster", "RS_Ratio"], ascending=[True, False])

for i in range(7):
    print(max_RS_Ratio[max_RS_Ratio["Cluster"] == i]["Variable"].values[0])

max_RS_Ratio

In [ ]:
clusters.info

In [ ]:
clusters.rsquare

In [ ]:
hierarchy = list()
variance_explained = dict()
for i in range(len(clusters.info), 0, -1):
    vch = VarClusHi(df7, maxeigval2=0.9, maxclus=i)
    vch.varclus()
    for _, x in vch.clusters.items(): # number, cluster
        cluster = frozenset(x.clus)
        hierarchy.append(cluster)
        variance_explained[cluster] = x.eigval1

plt.xticks(rotation='vertical')
cmap = plt.cm.tab20c(np.linspace(0, 1, len(hierarchy)))
for i, cluster in enumerate(hierarchy):
    x = list(cluster)
    y = np.ones(len(x)) * variance_explained[cluster]
    for bar in plt.bar(x, y, zorder=-i):
        bar.set_color(cmap[i])

plt.show()